In [1]:
import torch
import numpy as np 
from torch_geometric.data import Data
from torch.nn import Linear
import torch.nn.functional as F 
from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

ModuleNotFoundError: No module named 'torch'

In [ ]:
edge_index = torch.tensor([[0, 1, 1, 2, 3, 1],
                           [1, 0, 2, 1, 1, 3]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1], [1],[-1], [0], [1], [1],[-1], [0], [1], [1],[-1], [0], [1], [1]], dtype=torch.float)
y=torch.tensor([-1,0,1,1,-1,0,1,1,-1,0,1,1,-1,0,1,1])
torch.transpose(y,-1,0)
print(y)
data = Data(x=x,y=y, edge_index=edge_index)

data_test=Data(x=x, edge_index=edge_index)

In [ ]:
# Il faut définir : 
# data à partir de torchçgreometric 
# avec : 
# data.x = un vecteur de features de taille [Nombre noeuds, nombre de features] définit en torch.tensor()
# data.edge_index = un vecteur [2, nombre de liens], qui regroupent les liens entre les sommets de type torch.logn et défini en torch.tensor
# data.y=[num_nodes,1] en gros une seule sortie dans notre classe pour chaque noeud
#
# puis data= Datat(x=x, edge_index=edge_index, y=y)
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#

In [ ]:
embedding_size = 64

class GCN(torch.nn.Module):
    def __init__(self):
        # Init parent
        super(GCN, self).__init__()
        torch.manual_seed(42)

        # GCN layers
        self.initial_conv = GCNConv(data.num_features, embedding_size)
        self.conv1 = GCNConv(embedding_size, embedding_size)
        self.conv2 = GCNConv(embedding_size, embedding_size)
        self.conv3 = GCNConv(embedding_size, embedding_size)

        # Output layer
        self.out = Linear(embedding_size, 16)

    def forward(self, x, edge_index, batch_index):
        # First Conv layer
        hidden = self.initial_conv(x, edge_index)
        hidden = F.tanh(hidden)

        # Other Conv layers
        hidden = self.conv1(hidden, edge_index)
        hidden = F.tanh(hidden)
        hidden = self.conv2(hidden, edge_index)
        hidden = F.tanh(hidden)
        hidden = self.conv3(hidden, edge_index)
        hidden = F.tanh(hidden)
          
        # Global Pooling (stack different aggregations)
        #hidden = torch.cat([gmp(hidden, batch_index), 
                            #gap(hidden, batch_index)], dim=1)

        # Apply a final (linear) classifier.
        out = self.out(hidden)

        return out, hidden

model = GCN()
print(model)
print("Number of parameters: ", sum(p.numel() for p in model.parameters()))

In [ ]:
from torch_geometric.data import DataLoader
import warnings
warnings.filterwarnings("ignore")

# Root mean squared error
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0007)  

# Use GPU for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Wrap data in a data loader
data_size = len(data)
NUM_GRAPHS_PER_BATCH = 64
loader = DataLoader(data, 
                    batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
test_loader = DataLoader(data_test, 
                         batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)

def train(data):
    # Enumerate over the data
    
      # Use GPU
    data.to(device)  
      # Reset gradients
    optimizer.zero_grad() 
      # Passing the node features and the connection info
    pred, embedding = model(data.x.float(), data.edge_index, data.batch) 
      # Calculating the loss and gradients
    loss = torch.sqrt(loss_fn(pred, data.y))   
    print(loss)
    loss.backward()  
      # Update using the gradients
    optimizer.step()   
    return loss, embedding

print("Starting training...")
losses = []
for epoch in range(2000):
    loss, h = train(data)
    
    losses.append(loss)
    if epoch % 100 == 0:
      print(f"Epoch {epoch} | Train Loss {loss}")

In [ ]:
import pandas as pd 

# Analyze the results for one batch
test_batch = next(iter(test_loader))
with torch.no_grad():
    test_batch.to(device)
    pred, embed = model(test_batch.x.float(), test_batch.edge_index, test_batch.batch) 
    df = pd.DataFrame()
    df["y_real"] = test_batch.y.tolist()
    df["y_pred"] = pred.tolist()
df["y_real"] = df["y_real"].apply(lambda row: row[0])
df["y_pred"] = df["y_pred"].apply(lambda row: row[0])
df